In [1]:
import tensorflow as tf
import math
from tensorflow.examples.tutorials.mnist import input_data as mnist_data
print("Tensorflow version " + tf.__version__)
tf.set_random_seed(0)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Tensorflow version 1.9.0


In [2]:
LOGDIR = 'mnist_tutorial/'
mnist = mnist_data.read_data_sets(train_dir=LOGDIR + "data", one_hot=True, reshape=False, validation_size=0)

# input X: 28x28 grayscale images, the first dimension (None) will index the images in the mini-batch
X = tf.placeholder(tf.float32, [None, 28, 28, 1], name='X')
# correct answers will go here
Y_ = tf.placeholder(tf.float32, [None, 10], name='lable')

x_image = tf.reshape(X, [-1, 28, 28, 1])
tf.summary.image('input', x_image, 3)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist_tutorial/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist_tutorial/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting mnist_tutorial/data/t10k-images-idx3-ubyte.gz
Extracting mnist_tutorial/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


<tf.Tensor 'input:0' shape=() dtype=string>

In [4]:
# flatten the images into a single line of pixels
# -1 in the shape definition means "the only possible dimension that will preserve the number of elements"
XX = tf.reshape(X, [-1, 784])

lr = tf.placeholder(tf.float32)
# Probability of keeping a node during dropout = 1.0 at test time (no dropout) and 0.75 at training time
pkeep = tf.placeholder(tf.float32)

In [6]:
# five layers and their number of neurons (tha last layer has 10 softmax neurons)
L = 200
M = 100
N = 60
O = 30

In [7]:
# Weights initialised with small random values between -0.2 and +0.2
# When using RELUs, make sure biases are initialised with small *positive* values for example 0.1 = tf.ones([K])/10
def fc_layer(input, size_in, size_out, name="fc", act=tf.nn.relu):
  with tf.name_scope(name):
    w = tf.Variable(tf.truncated_normal([size_in, size_out], stddev=0.1), name="W")
    b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
    preact = tf.matmul(input, w) + b
    tf.summary.histogram("weights", w)
    tf.summary.histogram("biases", b)
    tf.summary.histogram(name + '/pre_activations', preact)
    if act==tf.nn.softmax:
        return preact
    activations = act(preact, 'activation')
    tf.summary.histogram(name + '/activations', activations)
    return activations

### This is our Network

In [8]:
# The model, with dropout at each layer

FC1 = fc_layer(XX, 28*28, L, "FC1")
FC1D = tf.nn.dropout(FC1, pkeep)
FC2 = fc_layer(FC1D, L, M, "FC2")
FC2D = tf.nn.dropout(FC2, pkeep)
FC3 = fc_layer(FC2D, M, N, "FC3")
FC3D = tf.nn.dropout(FC3, pkeep)
FC4 = fc_layer(FC3, N, O, "FC4")
FC4D = tf.nn.dropout(FC4, pkeep)
Ylogits = fc_layer(FC4D, O, 10, "FC5", act=tf.nn.softmax)
Y = tf.nn.softmax(Ylogits)

In [9]:
# cross-entropy loss function (= -sum(Y_i * log(Yi)) ), normalised for batches of 100  images
# TensorFlow provides the softmax_cross_entropy_with_logits function to avoid numerical stability
# problems with log(0) which is NaN
with tf.name_scope("cross_entropy"):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Y_)
    cross_entropy = tf.reduce_mean(cross_entropy)*100
    tf.summary.scalar("cross_entropy", cross_entropy)

# accuracy of the trained model, between 0 (worst) and 1 (best)
with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("accuracy", accuracy)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [10]:
with tf.name_scope("train"):
    train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy)

# summary merge
summ = tf.summary.merge_all()
# init
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
writer = tf.summary.FileWriter(LOGDIR + str(2.2))
writer.add_graph(sess.graph)

In [11]:
def training_step(i, update_test_data, update_train_data):

    # training on batches of 100 images with 100 labels
    batch_X, batch_Y = mnist.train.next_batch(100)

    # learning rate decay
    max_learning_rate = 0.003
    min_learning_rate = 0.0001
    decay_speed = 2000.0 # 0.003-0.0001-2000=>0.9826 done in 5000 iterations
    learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-i/decay_speed)

    # compute training values for visualisation
    if update_train_data:
        a, c, s = sess.run([accuracy, cross_entropy, summ], {X: batch_X, Y_: batch_Y, pkeep: 1.0})
        print(str(i) + ": accuracy:" + str(a) + " loss: " + str(c) + " (lr:" + str(learning_rate) + ")")
        writer.add_summary(s, i)
    # compute test values for visualisation
    if update_test_data:
        a, c, s = sess.run([accuracy, cross_entropy, summ], {X: mnist.test.images, Y_: mnist.test.labels, pkeep: 1.0})
        print(str(i) + ": ********* epoch " + str(i*100//mnist.train.images.shape[0]+1) + " ********* test accuracy:" + str(a) + " test loss: " + str(c))
        writer.add_summary(s, i)
    sess.run(train_step, {X: batch_X, Y_: batch_Y, pkeep: 0.75, lr: learning_rate})
    


In [12]:
for i in range(10000+1): training_step(i, i % 10 == 0, i % 100 == 0)

0: accuracy:0.16 loss: 229.68661 (lr:0.003)
0: ********* epoch 1 ********* test accuracy:0.1299 test loss: 230.15019
10: ********* epoch 1 ********* test accuracy:0.4801 test loss: 173.93202
20: ********* epoch 1 ********* test accuracy:0.7112 test loss: 94.97679
30: ********* epoch 1 ********* test accuracy:0.7629 test loss: 75.27462
40: ********* epoch 1 ********* test accuracy:0.8459 test loss: 56.38913
50: ********* epoch 1 ********* test accuracy:0.8638 test loss: 45.04578
60: ********* epoch 1 ********* test accuracy:0.8682 test loss: 43.445377
70: ********* epoch 1 ********* test accuracy:0.873 test loss: 42.76175
80: ********* epoch 1 ********* test accuracy:0.8931 test loss: 37.316513
90: ********* epoch 1 ********* test accuracy:0.8945 test loss: 36.255733
100: accuracy:0.94 loss: 17.582012 (lr:0.0028585653310520707)
100: ********* epoch 1 ********* test accuracy:0.9053 test loss: 31.13086
110: ********* epoch 1 ********* test accuracy:0.9079 test loss: 30.58337
120: ********

1010: ********* epoch 2 ********* test accuracy:0.9659 test loss: 11.711864
1020: ********* epoch 2 ********* test accuracy:0.9665 test loss: 11.720888
1030: ********* epoch 2 ********* test accuracy:0.9685 test loss: 11.621931
1040: ********* epoch 2 ********* test accuracy:0.9688 test loss: 11.297734
1050: ********* epoch 2 ********* test accuracy:0.968 test loss: 11.581103
1060: ********* epoch 2 ********* test accuracy:0.9671 test loss: 11.602927
1070: ********* epoch 2 ********* test accuracy:0.9641 test loss: 12.489472
1080: ********* epoch 2 ********* test accuracy:0.9664 test loss: 11.792702
1090: ********* epoch 2 ********* test accuracy:0.9636 test loss: 12.62788
1100: accuracy:0.96 loss: 9.705142 (lr:0.0017731544501034114)
1100: ********* epoch 2 ********* test accuracy:0.9667 test loss: 11.285608
1110: ********* epoch 2 ********* test accuracy:0.9656 test loss: 11.688295
1120: ********* epoch 2 ********* test accuracy:0.9661 test loss: 11.622091
1130: ********* epoch 2 ****

2020: ********* epoch 4 ********* test accuracy:0.9748 test loss: 8.554064
2030: ********* epoch 4 ********* test accuracy:0.9743 test loss: 9.387408
2040: ********* epoch 4 ********* test accuracy:0.9742 test loss: 8.927876
2050: ********* epoch 4 ********* test accuracy:0.9742 test loss: 8.761261
2060: ********* epoch 4 ********* test accuracy:0.9737 test loss: 8.900101
2070: ********* epoch 4 ********* test accuracy:0.9738 test loss: 8.8625
2080: ********* epoch 4 ********* test accuracy:0.9731 test loss: 9.138311
2090: ********* epoch 4 ********* test accuracy:0.9725 test loss: 9.292268
2100: accuracy:1.0 loss: 0.9514426 (lr:0.0011148194724223506)
2100: ********* epoch 4 ********* test accuracy:0.9718 test loss: 9.467469
2110: ********* epoch 4 ********* test accuracy:0.9727 test loss: 9.273138
2120: ********* epoch 4 ********* test accuracy:0.9739 test loss: 8.85366
2130: ********* epoch 4 ********* test accuracy:0.9748 test loss: 8.660856
2140: ********* epoch 4 ********* test ac

3040: ********* epoch 6 ********* test accuracy:0.9762 test loss: 8.146679
3050: ********* epoch 6 ********* test accuracy:0.977 test loss: 8.258582
3060: ********* epoch 6 ********* test accuracy:0.9769 test loss: 8.284016
3070: ********* epoch 6 ********* test accuracy:0.9761 test loss: 8.44173
3080: ********* epoch 6 ********* test accuracy:0.977 test loss: 8.162551
3090: ********* epoch 6 ********* test accuracy:0.9778 test loss: 8.068859
3100: accuracy:0.98 loss: 4.1905513 (lr:0.0007155191240975549)
3100: ********* epoch 6 ********* test accuracy:0.9771 test loss: 8.213913
3110: ********* epoch 6 ********* test accuracy:0.9766 test loss: 8.394674
3120: ********* epoch 6 ********* test accuracy:0.9773 test loss: 8.383512
3130: ********* epoch 6 ********* test accuracy:0.9764 test loss: 8.256912
3140: ********* epoch 6 ********* test accuracy:0.9767 test loss: 7.9956226
3150: ********* epoch 6 ********* test accuracy:0.9774 test loss: 7.9452157
3160: ********* epoch 6 ********* test

4050: ********* epoch 7 ********* test accuracy:0.9795 test loss: 7.733927
4060: ********* epoch 7 ********* test accuracy:0.9791 test loss: 7.7002516
4070: ********* epoch 7 ********* test accuracy:0.9796 test loss: 7.688669
4080: ********* epoch 7 ********* test accuracy:0.9803 test loss: 7.7359953
4090: ********* epoch 7 ********* test accuracy:0.98 test loss: 7.6766105
4100: accuracy:1.0 loss: 0.2066074 (lr:0.0004733312204046323)
4100: ********* epoch 7 ********* test accuracy:0.9793 test loss: 7.628789
4110: ********* epoch 7 ********* test accuracy:0.9789 test loss: 7.6119156
4120: ********* epoch 7 ********* test accuracy:0.9797 test loss: 7.5176992
4130: ********* epoch 7 ********* test accuracy:0.9794 test loss: 7.4953566
4140: ********* epoch 7 ********* test accuracy:0.9793 test loss: 7.5741396
4150: ********* epoch 7 ********* test accuracy:0.9794 test loss: 7.6587744
4160: ********* epoch 7 ********* test accuracy:0.9798 test loss: 7.4683824
4170: ********* epoch 7 *******

5060: ********* epoch 9 ********* test accuracy:0.9807 test loss: 7.2656326
5070: ********* epoch 9 ********* test accuracy:0.9799 test loss: 7.2914867
5080: ********* epoch 9 ********* test accuracy:0.9801 test loss: 7.3475947
5090: ********* epoch 9 ********* test accuracy:0.9797 test loss: 7.4427085
5100: accuracy:1.0 loss: 0.69449055 (lr:0.0003264368314033442)
5100: ********* epoch 9 ********* test accuracy:0.9796 test loss: 7.397168
5110: ********* epoch 9 ********* test accuracy:0.9801 test loss: 7.3175564
5120: ********* epoch 9 ********* test accuracy:0.98 test loss: 7.3120155
5130: ********* epoch 9 ********* test accuracy:0.9802 test loss: 7.349185
5140: ********* epoch 9 ********* test accuracy:0.9805 test loss: 7.385144
5150: ********* epoch 9 ********* test accuracy:0.9804 test loss: 7.415215
5160: ********* epoch 9 ********* test accuracy:0.9807 test loss: 7.3873615
5170: ********* epoch 9 ********* test accuracy:0.9807 test loss: 7.3910475
5180: ********* epoch 9 *******

6060: ********* epoch 11 ********* test accuracy:0.9813 test loss: 7.3490834
6070: ********* epoch 11 ********* test accuracy:0.9815 test loss: 7.366769
6080: ********* epoch 11 ********* test accuracy:0.9813 test loss: 7.3599606
6090: ********* epoch 11 ********* test accuracy:0.9815 test loss: 7.3559465
6100: accuracy:1.0 loss: 0.52698934 (lr:0.00023734088073430873)
6100: ********* epoch 11 ********* test accuracy:0.9811 test loss: 7.393638
6110: ********* epoch 11 ********* test accuracy:0.9804 test loss: 7.4073563
6120: ********* epoch 11 ********* test accuracy:0.9805 test loss: 7.401452
6130: ********* epoch 11 ********* test accuracy:0.9801 test loss: 7.461903
6140: ********* epoch 11 ********* test accuracy:0.98 test loss: 7.5028234
6150: ********* epoch 11 ********* test accuracy:0.9808 test loss: 7.536073
6160: ********* epoch 11 ********* test accuracy:0.9808 test loss: 7.51355
6170: ********* epoch 11 ********* test accuracy:0.9808 test loss: 7.461508
6180: ********* epoch 

7060: ********* epoch 12 ********* test accuracy:0.9812 test loss: 7.4284186
7070: ********* epoch 12 ********* test accuracy:0.9809 test loss: 7.458964
7080: ********* epoch 12 ********* test accuracy:0.9811 test loss: 7.486237
7090: ********* epoch 12 ********* test accuracy:0.9811 test loss: 7.4967966
7100: accuracy:1.0 loss: 0.8488221 (lr:0.00018330145499729437)
7100: ********* epoch 12 ********* test accuracy:0.9809 test loss: 7.5164466
7110: ********* epoch 12 ********* test accuracy:0.981 test loss: 7.537671
7120: ********* epoch 12 ********* test accuracy:0.9809 test loss: 7.536288
7130: ********* epoch 12 ********* test accuracy:0.9807 test loss: 7.5473404
7140: ********* epoch 12 ********* test accuracy:0.9804 test loss: 7.5275817
7150: ********* epoch 12 ********* test accuracy:0.9804 test loss: 7.5037813
7160: ********* epoch 12 ********* test accuracy:0.9802 test loss: 7.5652337
7170: ********* epoch 12 ********* test accuracy:0.98 test loss: 7.591196
7180: ********* epoch

8060: ********* epoch 14 ********* test accuracy:0.981 test loss: 7.5701
8070: ********* epoch 14 ********* test accuracy:0.9808 test loss: 7.5676837
8080: ********* epoch 14 ********* test accuracy:0.981 test loss: 7.561821
8090: ********* epoch 14 ********* test accuracy:0.9808 test loss: 7.605814
8100: accuracy:0.97 loss: 8.366529 (lr:0.0001505248864545312)
8100: ********* epoch 14 ********* test accuracy:0.9808 test loss: 7.5958805
8110: ********* epoch 14 ********* test accuracy:0.9806 test loss: 7.5554347
8120: ********* epoch 14 ********* test accuracy:0.9806 test loss: 7.502295
8130: ********* epoch 14 ********* test accuracy:0.9808 test loss: 7.436462
8140: ********* epoch 14 ********* test accuracy:0.9807 test loss: 7.4317174
8150: ********* epoch 14 ********* test accuracy:0.9801 test loss: 7.4304214
8160: ********* epoch 14 ********* test accuracy:0.9802 test loss: 7.4049435
8170: ********* epoch 14 ********* test accuracy:0.9807 test loss: 7.3937693
8180: ********* epoch 1

9060: ********* epoch 16 ********* test accuracy:0.9814 test loss: 7.429572
9070: ********* epoch 16 ********* test accuracy:0.9814 test loss: 7.450171
9080: ********* epoch 16 ********* test accuracy:0.9811 test loss: 7.4545555
9090: ********* epoch 16 ********* test accuracy:0.9808 test loss: 7.42843
9100: accuracy:0.99 loss: 3.7452064 (lr:0.00013064489271317272)
9100: ********* epoch 16 ********* test accuracy:0.9809 test loss: 7.4107714
9110: ********* epoch 16 ********* test accuracy:0.9809 test loss: 7.3863893
9120: ********* epoch 16 ********* test accuracy:0.9809 test loss: 7.3668613
9130: ********* epoch 16 ********* test accuracy:0.9811 test loss: 7.3737645
9140: ********* epoch 16 ********* test accuracy:0.9814 test loss: 7.365735
9150: ********* epoch 16 ********* test accuracy:0.9811 test loss: 7.371043
9160: ********* epoch 16 ********* test accuracy:0.9811 test loss: 7.381619
9170: ********* epoch 16 ********* test accuracy:0.9813 test loss: 7.3938556
9180: ********* epo